## Training Cluster

In [104]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [105]:
!pipeline cluster view

Current Cluster:
aws_k8s_training

Services:
NAME                           CLUSTER-IP     EXTERNAL-IP        PORT(S)                                                                                                                                                                                                                 AGE
airflow                        10.0.93.157    a9be74d74a86d...   80/TCP                                                                                                                                                                                                                  25d
airpal                         10.0.204.217   a9f022e58a86d...   80/TCP,81/TCP                                                                                                                                                                                                           25d
cassandra                      10.0.100.46    <nodes>            7199/TCP,9042/TCP,9160/TCP,7000/TCP

### [Kubernetes View of Training Cluster](http://kubernetes.demo.pipeline.io/)

In [106]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes.demo.pipeline.io">'
display(HTML(html))

## Scale Out Spark Worker

In [107]:
!pipeline service scale spark 3

replicationcontroller "spark-worker-2-0-1" scaled


In [108]:
!pipeline cluster view

Current Cluster:
aws_k8s_training

Services:
NAME                           CLUSTER-IP     EXTERNAL-IP        PORT(S)                                                                                                                                                                                                                 AGE
airflow                        10.0.93.157    a9be74d74a86d...   80/TCP                                                                                                                                                                                                                  25d
airpal                         10.0.204.217   a9f022e58a86d...   80/TCP,81/TCP                                                                                                                                                                                                           25d
cassandra                      10.0.100.46    <nodes>            7199/TCP,9042/TCP,9160/TCP,7000/TCP

In [109]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes.demo.pipeline.io">'
display(HTML(html))

### [Spark Admin](http://spark.demo.pipeline.io)

## Generate Spark ML Decision Tree

### Setup SparkSession

In [110]:
from pyspark.sql import SparkSession

sparkSession = SparkSession.builder.getOrCreate()

### Load Training Dataset from S3 into Spark

In [111]:
data = sparkSession.read.format("csv") \
  .option("inferSchema", "true").option("header", "true") \
  .load("s3a://datapalooza/R/census.csv")

data.head()

Row(age=39, workclass='State-gov', education='Bachelors', education_num=13, marital_status='Never-married', occupation='Adm-clerical', relationship='Not-in-family', race='White', sex='Male', capital_gain=2174, capital_loss=0, hours_per_week=40, native_country='United-States', income='<=50K')

### Build Spark ML Pipeline with Decision Tree Classifier

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier

formula = RFormula(formula = "income ~ .")
classifier = DecisionTreeClassifier()

pipeline = Pipeline(stages = [formula, classifier])

pipelineModel = pipeline.fit(data)

print(pipelineModel)

PipelineModel_43a2b98c0f47e3e99d94


In [22]:
print(pipelineModel.stages[1].toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_41aea0dc2ef384836421) of depth 5 with 59 nodes
  If (feature 23 in {0.0})
   If (feature 52 <= 7688.0)
    If (feature 22 <= 13.0)
     If (feature 54 <= 42.0)
      If (feature 53 <= 1876.0)
       Predict: 0.0
      Else (feature 53 > 1876.0)
       Predict: 0.0
     Else (feature 54 > 42.0)
      If (feature 9 in {0.0})
       Predict: 0.0
      Else (feature 9 not in {0.0})
       Predict: 0.0
    Else (feature 22 > 13.0)
     If (feature 54 <= 43.0)
      If (feature 0 <= 32.0)
       Predict: 0.0
      Else (feature 0 > 32.0)
       Predict: 0.0
     Else (feature 54 > 43.0)
      If (feature 0 <= 32.0)
       Predict: 0.0
      Else (feature 0 > 32.0)
       Predict: 0.0
   Else (feature 52 > 7688.0)
    If (feature 0 <= 20.0)
     If (feature 8 in {0.0})
      Predict: 0.0
     Else (feature 8 not in {0.0})
      Predict: 1.0
    Else (feature 0 > 20.0)
     If (feature 40 in {1.0})
      If (feature 0 <= 36.0)
       

## Convert Spark ML Pipeline to PMML

In [64]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sparkSession, data, pipelineModel)

print(pmmlBytes.decode("utf-8"))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2016-12-07T09:01:47Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="income" optype="categorical" dataType="string">
			<Value value="&lt;=50K"/>
			<Value value="&gt;50K"/>
		</DataField>
		<DataField name="education" optype="categorical" dataType="string">
			<Value value="HS-grad"/>
			<Value value="Some-college"/>
			<Value value="Bachelors"/>
			<Value value="Masters"/>
			<Value value="Assoc-voc"/>
			<Value value="11th"/>
			<Value value="Assoc-acdm"/>
			<Value value="10th"/>
			<Value value="7th-8th"/>
			<Value value="Prof-school"/>
			<Value value="9th"/>
			<Value value="12th"/>
			<Value value="Doctorate"/>
			<Value value="5th-6th"/>
			<Value value="1st-4th"/>
			<Value value="Preschool"/>
		</DataField>
		<DataField name="marital_status" optype="categorical" dataType="string">
			<Value value="Married-civ-sp

## Deployment Option 1:  Mutable Model Deployment

### AWS:  Deploy New Model to Live, Running Model Server

In [112]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


In [25]:
!pipeline cluster view

Current Cluster:
aws_k8s_predictions

Services:
NAME                    CLUSTER-IP     EXTERNAL-IP        PORT(S)           AGE
prediction-codegen      10.0.109.240   a58baa034b267...   80/TCP            12d
prediction-pmml         10.0.143.121   a49b9ff63b266...   80/TCP            12d
prediction-tensorflow   10.0.173.76    ab0dc9953b31c...   80/TCP            11d
turbine                 10.0.106.99    a3ee17de8b266...   80/TCP,8990/TCP   12d
weavescope-app          10.0.244.168   a337893b4b266...   80/TCP            12d

Replication Controller:
NAME                    DESIRED   CURRENT   READY     AGE
prediction-codegen      1         1         1         6d
prediction-pmml         1         1         1         7d
prediction-tensorflow   1         1         1         7d
turbine                 1         1         1         12d
weavescope-app          1         1         1         12d

Pods:
NAME                          READY     STATUS    RESTARTS   AGE
prediction-codegen-ctxq5      

### [Kubernetes View of Prediction Cluster - AWS](http://kubernetes-aws.demo.pipeline.io/)

In [113]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes-aws.demo.pipeline.io">'
display(HTML(html))

In [114]:
from urllib import request

update_url = 'http://prediction-pmml-aws.demo.pipeline.io/update-pmml/census'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = request.Request(update_url, headers=update_headers, data=pmmlBytes)
resp = request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


### GCP:  Deploy New Model to Live, Running Model Server

In [115]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


In [29]:
!pipeline cluster view

Current Cluster:
gcp_k8s_predictions

Services:
NAME                 CLUSTER-IP       EXTERNAL-IP      PORT(S)           AGE
prediction-codegen   10.179.246.129   104.198.6.242    80/TCP            12d
prediction-pmml      10.179.241.148   104.196.229.56   80/TCP            12d
turbine              10.179.253.175   104.198.108.42   80/TCP,8990/TCP   12d
weavescope-app       10.179.245.124   104.196.226.2    80/TCP            12d

Replication Controller:
NAME                    DESIRED   CURRENT   READY     AGE
prediction-codegen      1         1         1         6d
prediction-pmml         1         1         1         7d
prediction-tensorflow   1         1         1         7d
turbine                 1         1         1         12d
weavescope-app          1         1         1         12d

Pods:
NAME                          READY     STATUS    RESTARTS   AGE
prediction-codegen-k114g      1/1       Running   0          6d
prediction-pmml-uafkm         1/1       Running   0          

### [Kubernetes View of Prediction Cluster - GCP](http://kubernetes-gcp.demo.pipeline.io/)

In [30]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes-gcp.demo.pipeline.io">'
display(HTML(html))

In [116]:
from urllib import request

update_url = 'http://prediction-pmml-gcp.demo.pipeline.io/update-pmml/census'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = request.Request(update_url, headers=update_headers, data=pmmlBytes)
resp = request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


## Predict on New Data

### AWS

In [117]:
from urllib import request

evaluate_url = 'http://prediction-pmml-aws.demo.pipeline.io/evaluate-pmml/census'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
input_params = '{"age":39,"workclass":"State-gov","education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}' 
encoded_input_params = input_params.encode('utf-8')

req = request.Request(evaluate_url, headers=evaluate_headers, data=encoded_input_params)
resp = request.urlopen(req)

print(resp.read()) # Should return valid classification with probabilities

b'{"results":[[{\'income\': \'NodeScoreDistribution{result=<=50K, probability_entries=[<=50K=0.9791280929136509, >50K=0.0208719070863491], entityId=6, confidence_entries=[]}\'}]]'


### GCP

In [118]:
from urllib import request
import json

evaluate_url = 'http://prediction-pmml-gcp.demo.pipeline.io/evaluate-pmml/census'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
input_params = '{"age":39,"workclass":"State-gov","education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}' 
encoded_input_params = input_params.encode('utf-8')

req = request.Request(evaluate_url, headers=evaluate_headers, data=encoded_input_params)
resp = request.urlopen(req)

print(resp.read()) # Should return valid classification with probabilities

b'{"results":[[{\'income\': \'NodeScoreDistribution{result=<=50K, probability_entries=[<=50K=0.9791280929136509, >50K=0.0208719070863491], entityId=6, confidence_entries=[]}\'}]]'


## Load Test Predictions Across AWS and Google Cloud

In [119]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [120]:
!pipeline service deploy loadtest

...Starting Load Test...
replicationcontroller "loadtest" created


### [View of Prediction Services](http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Predictions%20-%20AWS%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-aws.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%2C%7B%22name%22%3A%22Predictions%20-%20GCP%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-gcp.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D)

In [68]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Predictions%20-%20AWS%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-aws.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%2C%7B%22name%22%3A%22Predictions%20-%20GCP%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-gcp.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D">'
display(HTML(html))

## Scale Out Load Test

In [71]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [73]:
!pipeline service scale loadtest 3

replicationcontroller "loadtest" scaled


## Scale Out Model Prediction Servers

### AWS

In [77]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


In [78]:
!pipeline service scale prediction 2

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled
Error from server: replicationcontrollers "prediction-cache" not found
replicationcontroller "prediction-tensorflow" scaled


In [79]:
!pipeline cluster view

Current Cluster:
aws_k8s_predictions

Services:
NAME                    CLUSTER-IP     EXTERNAL-IP        PORT(S)           AGE
prediction-codegen      10.0.109.240   a58baa034b267...   80/TCP            12d
prediction-pmml         10.0.143.121   a49b9ff63b266...   80/TCP            12d
prediction-tensorflow   10.0.173.76    ab0dc9953b31c...   80/TCP            11d
turbine                 10.0.106.99    a3ee17de8b266...   80/TCP,8990/TCP   12d
weavescope-app          10.0.244.168   a337893b4b266...   80/TCP            12d

Replication Controller:
NAME                    DESIRED   CURRENT   READY     AGE
prediction-codegen      2         2         1         6d
prediction-pmml         2         2         1         7d
prediction-tensorflow   2         2         1         7d
turbine                 1         1         1         12d
weavescope-app          1         1         1         12d

Pods:
NAME                          READY     STATUS              RESTARTS   AGE
prediction-codegen-c

### GCP

In [80]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


In [81]:
!pipeline service scale prediction 2

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled
Error from server: replicationcontrollers "prediction-cache" not found
replicationcontroller "prediction-tensorflow" scaled


In [43]:
!pipeline cluster view

Current Cluster:
aws_k8s_predictions

Services:
NAME                    CLUSTER-IP     EXTERNAL-IP        PORT(S)           AGE
prediction-codegen      10.0.109.240   a58baa034b267...   80/TCP            12d
prediction-pmml         10.0.143.121   a49b9ff63b266...   80/TCP            12d
prediction-tensorflow   10.0.173.76    ab0dc9953b31c...   80/TCP            11d
turbine                 10.0.106.99    a3ee17de8b266...   80/TCP,8990/TCP   12d
weavescope-app          10.0.244.168   a337893b4b266...   80/TCP            12d

Replication Controller:
NAME                    DESIRED   CURRENT   READY     AGE
prediction-codegen      2         2         1         6d
prediction-pmml         2         2         1         7d
prediction-tensorflow   2         2         1         7d
turbine                 1         1         1         12d
weavescope-app          1         1         1         12d

Pods:
NAME                          READY     STATUS              RESTARTS   AGE
prediction-codegen-1

## Deployment Option 2:  Immutable Model Deployment

### Save Model to Disk

In [44]:
with open('/root/census.pmml', 'wb') as f:
  f.write(pmmlBytes)

!cat /root/census.pmml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2016-12-07T08:45:46Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="income" optype="categorical" dataType="string">
			<Value value="&lt;=50K"/>
			<Value value="&gt;50K"/>
		</DataField>
		<DataField name="education" optype="categorical" dataType="string">
			<Value value="HS-grad"/>
			<Value value="Some-college"/>
			<Value value="Bachelors"/>
			<Value value="Masters"/>
			<Value value="Assoc-voc"/>
			<Value value="11th"/>
			<Value value="Assoc-acdm"/>
			<Value value="10th"/>
			<Value value="7th-8th"/>
			<Value value="Prof-school"/>
			<Value value="9th"/>
			<Value value="12th"/>
			<Value value="Doctorate"/>
			<Value value="5th-6th"/>
			<Value value="1st-4th"/>
			<Value value="Preschool"/>
		</DataField>
		<DataField name="marital_status" optype="categorical" dataType="string">
			<Value value="Married-civ-sp

### Commit to Github and Trigger Canary Model Deployment

### Monitor Canary Model Deployment

In [45]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://airflow.demo.pipeline.io">'
display(HTML(html))

## Scale In and Cleanup

### Load Test

In [3]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [4]:
!pipeline service undeploy loadtest

...Loadtest...
...Ignore Any Errors...
replicationcontroller "loadtest" deleted


### Spark

In [5]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [6]:
!pipeline service scale spark 1

replicationcontroller "spark-worker-2-0-1" scaled


In [7]:
!pipeline cluster view

Current Cluster:
aws_k8s_training

Services:
NAME                           CLUSTER-IP     EXTERNAL-IP        PORT(S)                                                                                                                                                                                                                 AGE
airflow                        10.0.93.157    a9be74d74a86d...   80/TCP                                                                                                                                                                                                                  25d
airpal                         10.0.204.217   a9f022e58a86d...   80/TCP,81/TCP                                                                                                                                                                                                           25d
cassandra                      10.0.100.46    <nodes>            7199/TCP,9042/TCP,9160/TCP,7000/TCP

### AWS Predictions

In [8]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


In [9]:
!pipeline service scale prediction 1

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled


In [10]:
!pipeline cluster view

Current Cluster:
aws_k8s_predictions

Services:
NAME                    CLUSTER-IP     EXTERNAL-IP        PORT(S)           AGE
prediction-codegen      10.0.109.240   a58baa034b267...   80/TCP            12d
prediction-pmml         10.0.143.121   a49b9ff63b266...   80/TCP            12d
prediction-tensorflow   10.0.173.76    ab0dc9953b31c...   80/TCP            11d
turbine                 10.0.106.99    a3ee17de8b266...   80/TCP,8990/TCP   12d
weavescope-app          10.0.244.168   a337893b4b266...   80/TCP            12d

Replication Controller:
NAME                    DESIRED   CURRENT   READY     AGE
prediction-codegen      1         1         1         6d
prediction-pmml         1         1         1         7d
prediction-tensorflow   2         2         2         7d
turbine                 1         1         1         12d
weavescope-app          1         1         1         12d

Pods:
NAME                          READY     STATUS    RESTARTS   AGE
prediction-codegen-ctxq5      

### GCP Predictions

In [11]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


In [12]:
!pipeline service scale prediction 1

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled


In [13]:
!pipeline cluster view

Current Cluster:
gcp_k8s_predictions

Services:
NAME                 CLUSTER-IP       EXTERNAL-IP      PORT(S)           AGE
prediction-codegen   10.179.246.129   104.198.6.242    80/TCP            12d
prediction-pmml      10.179.241.148   104.196.229.56   80/TCP            12d
turbine              10.179.253.175   104.198.108.42   80/TCP,8990/TCP   12d
weavescope-app       10.179.245.124   104.196.226.2    80/TCP            12d

Replication Controller:
NAME                 DESIRED   CURRENT   READY     AGE
prediction-codegen   1         1         1         6d
prediction-pmml      1         1         1         7d
turbine              1         1         1         12d
weavescope-app       1         1         1         12d

Pods:
NAME                       READY     STATUS    RESTARTS   AGE
prediction-codegen-k114g   1/1       Running   0          6d
prediction-pmml-uafkm      1/1       Running   0          7d
turbine-u6588              1/1       Running   0          12d
weavescope-app-n